# Micrograd 

Este notebook está basado en el video **"[The spelled-out intro to neural networks and backpropagation: building micrograd](https://youtu.be/VMj-3S1tku0?si=VUGvwhSh3fSfze5d)"**, donde explica el **backpropagation** (la mejor clase práctica, oro puro!).